# After Bye


In [38]:
# Dependencies and Setup
from pandasql import sqldf
import sqlite3
from sqlite3 import Error
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
import random
from datetime import datetime
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import text

In [39]:
from config import user, password, host, port, db
print(port)

5433


In [40]:
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}")

In [41]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT count(*) FROM main"))
    for row in result:
        print(row)

(3664,)


In [42]:
with engine.connect() as connection:
    result = connection.execute(text("SELECT count(*) FROM bye_weeks"))
    for row in result:
        print(row)

(921,)


In [43]:
main_df = pd.read_sql_query('select * from main',con=engine)

In [44]:
main_df

,game_id,date,season,week,day,away_team,away_score,away_spread_open,away_spread_open_odds,away_spread_close,...,home_team,home_score,home_spread_open,home_spread_open_odds,home_spread_close,home_spread_close_odds,home_moneyline_open,home_moneyline_close,total_open,total_close
0,66,2009-09-10,2009,2,Thu,Tennessee,10,5.5,-110.0,6.5,...,Pittsburgh,13,-5.5,-110.0,-6.5,107.0,-210.0,-260.0,35.0,35.5
1,67,2009-09-13,2009,2,Sun,Miami,7,4.0,-110.0,4.0,...,Atlanta,19,-4.0,-110.0,-4.0,-104.0,-215.0,-210.0,43.0,44.0
2,68,2009-09-13,2009,2,Sun,Kansas City,24,8.5,-110.0,13.0,...,Baltimore,38,-8.5,-110.0,-13.0,104.0,-600.0,-700.0,37.5,36.5
3,69,2009-09-13,2009,2,Sun,Philadelphia,38,1.0,-110.0,-1.0,...,Carolina,10,-1.0,-110.0,1.0,111.0,110.0,115.0,44.5,43.5
4,70,2009-09-13,2009,2,Sun,Denver,12,1.0,-110.0,4.5,...,Cincinnati,7,-1.0,-110.0,-4.5,-104.0,-215.0,-222.0,44.5,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3659,4721292,2023-10-22,2023,7,Sun,Arizona,10,7.0,-110.0,8.5,...,Seattle,20,-7.0,-110.0,-8.5,-110.0,-330.0,-405.0,46.5,43.5
3660,4721293,2023-10-22,2023,7,Sun,L.A. Chargers,17,6.0,-110.0,5.5,...,Kansas City,31,-6.0,-110.0,-5.5,-115.0,-270.0,-260.0,52.0,48.5
3661,4721294,2023-10-22,2023,7,Sun,Green Bay,17,-1.5,-102.0,1.0,...,Denver,19,1.5,-120.0,-1.0,-110.0,-106.0,-118.0,46.5,44.0
3662,4721295,2023-10-22,2023,7,Sun,Miami,17,2.5,-105.0,3.0,...,Philadelphia,31,-2.5,-115.0,-3.0,-104.0,-142.0,-154.0,51.5,51.5


In [45]:
bye_weeks_df = pd.read_sql_query('select * from bye_weeks',con=engine)

In [46]:
bye_weeks_df

,season,week,team
0,2009,4,Arizona
1,2009,4,Atlanta
2,2009,4,Carolina
3,2009,4,Philadelphia
4,2009,5,Chicago
...,...,...,...
916,2023,7,Pittsburgh
917,2023,7,San Francisco
918,2023,7,Seattle
919,2023,7,Tampa Bay


In [47]:
results_df = (sqldf('''SELECT a.game_id
                    , a.season
                    , a.week
                    , a.day
                    , a.away_team
                    , 'W' as away_team_result
                    , a.away_score
                    , a.away_spread_close as spread_close
                    , a.home_score
                    , 'L' as home_team_result
                    , a.home_team
                    , a.away_score + a.away_spread_close - a.home_score as win_margin
                FROM main_df as a
                WHERE ( a.away_score + a.away_spread_close > a.home_score + 0)
                UNION
                SELECT a.game_id
                    , a.season
                    , a.week
                    , a.day
                    , a.away_team
                    , 'L' as away_team_result
                    , a.away_score
                    , a.away_spread_close
                    , a.home_score
                    , 'W' as home_team_result
                    , a.home_team
                    , a.home_score - a.away_spread_close - a.away_score
                FROM main_df as a
                WHERE ( a.home_score - a.away_spread_close > a.away_score + 0)
                UNION
                SELECT a.game_id
                    , a.season
                    , a.week
                    , a.day
                    , a.away_team
                    , 'P' as away_team_result
                    , a.away_score
                    , a.away_spread_close
                    , a.home_score
                    , 'P' as home_team_result
                    , a.home_team
                    , 0.0
                FROM main_df as a
                WHERE ( a.home_score - a.away_spread_close = a.away_score + 0)
                ORDER BY a.game_id
            '''))
results_df

,game_id,season,week,day,away_team,away_team_result,away_score,spread_close,home_score,home_team_result,home_team,win_margin
0,66,2009,2,Thu,Tennessee,W,10,6.5,13,L,Pittsburgh,3.5
1,67,2009,2,Sun,Miami,L,7,4.0,19,W,Atlanta,8.0
2,68,2009,2,Sun,Kansas City,L,24,13.0,38,W,Baltimore,1.0
3,69,2009,2,Sun,Philadelphia,W,38,-1.0,10,L,Carolina,27.0
4,70,2009,2,Sun,Denver,W,12,4.5,7,L,Cincinnati,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...
3659,4721292,2023,7,Sun,Arizona,L,10,8.5,20,W,Seattle,1.5
3660,4721293,2023,7,Sun,L.A. Chargers,L,17,5.5,31,W,Kansas City,8.5
3661,4721294,2023,7,Sun,Green Bay,L,17,1.0,19,W,Denver,1.0
3662,4721295,2023,7,Sun,Miami,L,17,3.0,31,W,Philadelphia,11.0


In [48]:
print(f"results_df row count: {len(results_df)}")

results_df row count: 3664


In [49]:
results_df.to_excel('./Output/results.xlsx')

In [50]:
bye_weeks_df

,season,week,team
0,2009,4,Arizona
1,2009,4,Atlanta
2,2009,4,Carolina
3,2009,4,Philadelphia
4,2009,5,Chicago
...,...,...,...
916,2023,7,Pittsburgh
917,2023,7,San Francisco
918,2023,7,Seattle
919,2023,7,Tampa Bay


In [76]:
# DECLARE @query_title varchar(40)
# set @query_title = 'Away Favs After a Bye, Home played last week'
# query_title <- "virginica"
query_title = 'Away Favs After a Bye, Home played last week'
print(query_title)

temp_df = (sqldf('''SELECT '$query_title' as query
                        , count(*) as count, "W" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.away_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.spread_close < 0
                    AND r.away_team_result = "W"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.home_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    UNION
                    SELECT 'x'
                        , count(*) as count, "L" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.away_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.spread_close < 0
                    AND r.away_team_result = "L"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.home_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    ORDER BY 1, 3 DESC
                    '''))
# First instance of display dataframe
display_df = temp_df.copy()
display_df

Away Favs After a Bye, Home played last week


,query,count,result
0,$query_title,27,W
1,x,30,L


In [37]:
print('Away Dogs After a Bye, Home played last week')

temp_df = (sqldf('''SELECT count(*) as count, "W" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.away_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.spread_close > 0
                    AND r.away_team_result = "W"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.home_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    UNION
                    SELECT count(*) as count, "L" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.away_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.spread_close > 0
                    AND r.away_team_result = "L"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.home_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

temp_df

Away Dogs After a Bye, Home played last week


,Count,Result
0,65,W
1,53,L


In [30]:
print('Home Favs After a Bye, Away played last week')

test_df = (sqldf('''SELECT count(*) as count, "W" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.home_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.spread_close > 0
                    AND r.home_team_result = "W"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.away_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    UNION
                    SELECT count(*) as count, "L" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.home_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.spread_close > 0
                    AND r.home_team_result = "L"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.away_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

test_df

Home Favs after a bye, Away played last week


,Count,Result
0,73,W
1,68,L


In [38]:
print('Home Dogs After a Bye, Away played last week')

temp_df = (sqldf('''SELECT count(*) as count, "W" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.home_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.spread_close < 0
                    AND r.home_team_result = "W"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.away_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    UNION
                    SELECT count(*) as count, "L" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.home_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.spread_close < 0
                    AND r.home_team_result = "L"
                    AND NOT EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.away_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

temp_df

Home Dogs After a Bye, Away played last week


,Count,Result
0,36,W
1,49,L


In [39]:
print('Favs, Both After a Bye')

temp_df = (sqldf('''SELECT count(*) as count, "W" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.season = b.season
                    AND r.week = b.week - 1
                    WHERE ( 
                            (r.away_team = b.team AND r.spread_close < 0 AND r.away_team_result = "W")
                            OR
                            (r.home_team = b.team AND r.spread_close > 0 AND r.home_team_result = "W")
                            )
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.home_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b3
                                     WHERE r.away_team = b3.team
                                     AND r.season = b3.season
                                     AND r.week = b3.week - 1
                                    )
                    UNION
                    SELECT count(*) as count, "L" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.season = b.season
                    AND r.week = b.week - 1
                    WHERE ( 
                            (r.away_team = b.team AND r.spread_close < 0 AND r.away_team_result = "L")
                            OR
                            (r.home_team = b.team AND r.spread_close > 0 AND r.home_team_result = "L")
                            )
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.home_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b3
                                     WHERE r.away_team = b3.team
                                     AND r.season = b3.season
                                     AND r.week = b3.week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

temp_df

Favs, Both After a Bye


,Count,Result
0,123,W
1,125,L


In [40]:
print('Away, Both After a Bye')

temp_df = (sqldf('''SELECT count(*) as count, "W" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.away_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.away_team_result = "W"
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.home_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b3
                                     WHERE r.away_team = b3.team
                                     AND r.season = b3.season
                                     AND r.week = b3.week - 1
                                    )
                    UNION
                    SELECT count(*) as count, "L" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.away_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.away_team_result = "L"
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.home_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b3
                                     WHERE r.away_team = b3.team
                                     AND r.season = b3.season
                                     AND r.week = b3.week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

temp_df

Away, Both After a Bye


,Count,Result
0,123,W
1,125,L


In [41]:
print('Home Favs, After after a Bye')

temp_df = (sqldf('''SELECT count(*) as count, "W" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.home_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.spread_close > 0
                    AND r.home_team_result = "W"
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.home_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b3
                                     WHERE r.away_team = b3.team
                                     AND r.season = b3.season
                                     AND r.week = b3.week - 1
                                    )
                    UNION
                    SELECT count(*) as count, "L" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.home_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.spread_close > 0
                    AND r.home_team_result = "L"
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.home_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b3
                                     WHERE r.away_team = b3.team
                                     AND r.season = b3.season
                                     AND r.week = b3.week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

temp_df

Home Favs, After after a Bye


,Count,Result
0,80,W
1,80,L


In [49]:
print('Home Dogs, After after a Bye')

temp_df = (sqldf('''SELECT count(*) as count, "W" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.home_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.spread_close < 0
                    AND r.home_team_result = "W"
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.home_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b3
                                     WHERE r.away_team = b3.team
                                     AND r.season = b3.season
                                     AND r.week = b3.week - 1
                                    )
                    UNION
                    SELECT count(*) as count, "L" as result
                    FROM results_df as r
                    INNER JOIN bye_weeks_df as b
                    ON r.home_team = b.team
                    AND r.season = b.season
                    AND r.week = b.week - 1
                    WHERE r.spread_close < 0
                    AND r.home_team_result = "L"
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b2
                                     WHERE r.home_team = b2.team
                                     AND r.season = b2.season
                                     AND r.week = b2.week - 1
                                    )
                    AND EXISTS ( SELECT "Y"
                                     FROM bye_weeks_df as b3
                                     WHERE r.away_team = b3.team
                                     AND r.season = b3.season
                                     AND r.week = b3.week - 1
                                    )
                    ORDER BY 2 DESC
                    '''))

display_df = pd.concat([display_df, temp_df])
display_df

Home Dogs, After after a Bye


,Count,Result
0,27,W
1,30,L
0,45,W
1,43,L
